In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras import layers
from keras import models
from keras import optimizers
import datetime

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator, array_to_img

Using TensorFlow backend.


In [2]:
train_dir = '../../data/train'
validation_dir = '../../data/dev'
test_dir = '../../data/test'

In [3]:
# Your code here; add fully connected layers on top of the convolutional base

# Initialize Base
cnn_base = VGG19(weights='imagenet',
                 include_top=False,
                 input_shape=(240, 240, 3))

# Define Model Architecture
model = models.Sequential()
model.add(cnn_base)
model.add(layers.Flatten())
model.add(layers.Dense(64, kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dense(128, kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dense(256, kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dense(128, kernel_regularizer=l2(0.001), activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

cnn_base.trainable = False

# You can check whether a layer is trainable (or alter its setting) through the layer.trainable attribute
for layer in model.layers:
    print(layer.name, layer.trainable)
    
# Similarly, we can check how many trainable weights are in the model 
print(len(model.trainable_weights))

model.summary()

80142336/80134624 [==============================] - 7s 0us/step
vgg19 False
flatten_1 True
dense_1 True
dense_2 True
dense_3 True
dense_4 True
dense_5 True
10
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1605696   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)           

In [4]:
original_start = datetime.datetime.now()
start = datetime.datetime.now()

# Define Initial Parameters (same as previous code block)
datagen = ImageDataGenerator(rescale=1./255) 
batch_size = 10

# Get all the data in the directory split/train (542 images), and reshape them
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')


test_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')



train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(240, 240), 
                                                    batch_size= 20,
                                                    class_mode='categorical') 

# Get all the data in the directory split/validation, and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(validation_dir, d
                                                                       target_size=(240, 240), 
                                                                       batch_size=20,
                                                                       class_mode='categorical')

# Get all the data in the directory split/test, and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir, 
                                                                        target_size=(240, 240), 
                                                                        batch_size=180,
                                                                        class_mode='categorical',
                                                                        shuffle=False)

        
test_images, test_labels = next(test_generator)


# Compilation
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

# Fitting the Model
history = model.fit_generator(train_generator,
                              steps_per_epoch=8,
                              epochs=4,
                              validation_data=val_generator,
                              validation_steps=10)


end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

Found 12422 images belonging to 7 classes.
Found 1458 images belonging to 7 classes.
Found 3147 images belonging to 7 classes.
Epoch 1/4
8/8 [==============================] - 465s 58s/step - loss: 1.9896 - acc: 0.1312 - val_loss: 2.0191 - val_acc: 0.1150
Epoch 2/4
8/8 [==============================] - 516s 65s/step - loss: 1.9701 - acc: 0.0688 - val_loss: 1.9804 - val_acc: 0.1400
Epoch 3/4
8/8 [==============================] - 2482s 310s/step - loss: 1.9491 - acc: 0.0938 - val_loss: 1.9354 - val_acc: 0.1450
Epoch 4/4
7/8 [=========================>....] - ETA: 1:42 - loss: 1.9486 - acc: 0.1357

KeyboardInterrupt: 